In [ ]:
import pandas as pd

# Load CSV files
facts_df = pd.read_csv('facts_table.csv')
sales_dim_df = pd.read_csv('sales_dim.csv')
property_dim_df = pd.read_csv('property_dim.csv')
location_dim_df = pd.read_csv('location_dim.csv')
datetime_dim_df = pd.read_csv('datetime_dim.csv')

# Join the tables
big_table = pd.merge(facts_df, sales_dim_df, on='sales_id_fact')
big_table = pd.merge(big_table, property_dim_df, on='property_id')
big_table = pd.merge(big_table, location_dim_df, on='location_id')
big_table = pd.merge(big_table, datetime_dim_df, on='datetime_id')

# Save denormalized data to a CSV file
big_table.to_csv('denormalized_data.csv', index=False)


## Dumping data into RDS : 

In [1]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import mysql.connector

# MySQL database connection parameters
db_config = {
    user: 'root',
    host: 'my-rds-instance.us-west-2.rds.amazonaws.com',
    database: 'mydb',
    port: 3306  
}

# Connect to MySQL database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Load data into MySQL table
with open('denormalized_data.csv', 'r') as file:
    next(file)  # Skip header
    cursor.execute('CREATE TABLE IF NOT EXISTS denormalized_table (sales_serial_number INT, ...)')
    cursor.copy_from(file, 'denormalized_table', sep=',')

# Commit changes and close connection
conn.commit()
conn.close()


In [2]:
## Performing SQL case Study : 

In [ ]:
# Calculate the average sales amount for each month in the year 2023. 
query1 = """
SELECT residental_type, SUM(sales_amount) AS total_sales_amount
FROM denormalized_table
GROUP BY residental_type
"""
cursor.execute(query1)
result1 = cursor.fetchall()
print("Query 1:")
for row in result1:
    print(row)

In [ ]:
# Find the top 5 property towns with the highest average assessed value.
query2 = """
SELECT property_town, AVG(assessed_value) AS avg_assessed_value
FROM denormalized_table
GROUP BY property_town
ORDER BY avg_assessed_value DESC
LIMIT 5;

"""
cursor.execute(query2)


In [ ]:
# Identify the top 5 property towns with the highest sales-to-assessed value ratio.
query3 = """
SELECT property_town, SUM(sales_amount) / SUM(assessed_value) AS sales_to_assessed_value_ratio
FROM denormalized_table
GROUP BY property_town
ORDER BY sales_to_assessed_value_ratio DESC
LIMIT 5;
"""
cursor.execute(query3)


In [ ]:
# Identify the sales day of the week with the highest total sales ratio.
query4 = """
SELECT sales_weekday, SUM(sales_ratio) AS total_sales_ratio
FROM denormalized_table
GROUP BY sales_weekday
ORDER BY total_sales_ratio DESC
LIMIT 1;

"""
cursor.execute(query4)
